In [1]:
import pandas as pd
import regex as re
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import lightgbm as lgbm
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import pickle
# loading the tensorflow universal sentence encoder 4 model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2021-11-18 13:01:47.162498: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# set dataframe display settings to show 1000 rows
pd.options.display.max_rows = 1000

In [3]:
# prep a data set with a column named 'cleanText' for model prediction
def prepDataForTextModel(predictionData):
    X_train = []
    for r in tqdm(predictionData.cleanText.values):
        emb = embed([r])
        review_emb = tf.reshape(emb, [-1]).numpy()
        X_train.append(review_emb)
    predictionData = np.array(X_train)
    return predictionData

In [4]:
# prep a dataset containing the columns ['emojiCount', 'followers', 'following','hashtagCount', 'urlCount',
# 'tweetCount', 'verified'] which are the required features for prediction
def prepDataForModel(predictionData):
    predictionData = predictionData[['emojiCount', 'followers', 'following', 'hashtagCount', 'urlCount', 'tweetCount', 'verified']]
    return predictionData

In [5]:
# creates trainingSet from human and bot dataset
def munge():
    # read bot and human datasets
    trainHumans = pd.read_csv('trainingDataHuman.csv')
    trainBots = pd.read_csv('trainingDataBots.csv')
    
    # Combine bot and human dataframes to create full trainingSet, then schuffle them
    frames = [trainHumans, trainBots]
    trainingSet = pd.concat(frames)
    trainingSet = trainingSet.sample(frac=1).reset_index(drop=True)
    
    # replacing na values in trainingSet with 'empty'
    trainingSet["cleanText"].fillna("empty", inplace = True)
    # changing true and false to 1 and 0
    trainingSet.verified = trainingSet.verified.replace({True: 1, False: 0})
    
    # embed clean text
    trainingSet['embededText'] = trainingSet.apply(lambda row : tf.reshape(embed([row["cleanText"]]),[-1]).numpy(), axis = 1)
    
    return trainingSet

In [20]:
# gets performance metrics from any set of true and predicted values
def getMetrics(y_true, y_pred):
    print(confusion_matrix(y_true, y_pred))
    print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
    print("Precision: " + str(precision_score(y_true, y_pred)))
    print("Recall: " + str(recall_score(y_true, y_pred)))

In [7]:
# trains a text model given a training dataset
def trainTextModel(trainingSet):
    print(trainingSet.columns)
    X_train = []
    for r in tqdm(trainingSet.cleanText.values):
        emb = embed([r])
        review_emb = tf.reshape(emb, [-1]).numpy()
        X_train.append(review_emb)

    X_train = np.array(X_train)
    y_train = trainingSet.human.values

    validationSet = X_train[1846:]
    validation_y = trainingSet.iloc[1846:]
    X_train = X_train[0:1845]
    y_train = y_train[0:1845]
    
    # split data into train and testing data
    X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.2, random_state = 42)
    
    # get the train and test data for the training sequence
    train_data = lgbm.Dataset(X_train, label=y_train)
    test_data = lgbm.Dataset(X_test, label=y_test)

    # parameters we'll use for the prediction and pass to lightgbm model
    parameters = {
        'metric': 'auc',
        'boosting': 'dart',
        'num_leaves': 31,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.5,
        'bagging_freq': 20,
        'learning_rate': 0.05,
        'verbose': 0
    }
    parameters,
    # train the classifier
    classifier = lgbm.train(parameters,
                            train_data,
                            valid_sets= test_data,
                            num_boost_round=1000,
                            early_stopping_rounds=100)
    
    # PREDICTION - Make prediction on validation dataset
    val_pred = classifier.predict(validationSet)

    # Insert predictions into validation dataframe
    validation_y.insert(1, "Pred", val_pred, True)
    
    # Round continuous predictions into true or false (0 or 1)
    validation_y['Pred'] = validation_y['Pred'].apply(lambda x: round(x))
    
    y_true = validation_y['human']
    y_pred = validation_y['Pred']
    
    getMetrics(y_true, y_pred)
    
    return classifier

In [8]:
# trains and returns model which accepts the following features
# 'emojiCount', 'followers', 'following', 'hashtagCount', 'urlCount', 'tweetCount', 'verified'
def trainModel(trainingSet):
    validationSet = trainingSet.tail(300)
    validation_y = trainingSet.tail(300)
    
    validationSet.drop(columns=['id', 'rawText', 'cleanText', 'human', 'embededText'], inplace=True)
    
    X_train = trainingSet.head(len(trainingSet)-300)
    y_train = trainingSet.head(len(trainingSet)-300).human.values
    
    X_train.drop(columns=['id', 'rawText', 'cleanText', 'human', 'embededText'], inplace=True)
    
    print(X_train.columns)
    # split data into train and testing data
    X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.2, random_state = 42)
    
    # get the train and test data for the training sequence
    train_data = lgbm.Dataset(X_train, label=y_train)
    test_data = lgbm.Dataset(X_test, label=y_test)

    # parameters we'll use for the prediction and pass to lightgbm model
    parameters = {
        'metric': 'auc',
        'boosting': 'dart',
        'num_leaves': 31,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.5,
        'bagging_freq': 20,
        'learning_rate': 0.05,
        'verbose': 0
    }
    parameters,
    # train the classifier
    classifier = lgbm.train(parameters,
                            train_data,
                            valid_sets= test_data,
                            num_boost_round=1000,
                            early_stopping_rounds=100)
    
    # PREDICTION - Make prediction on validation dataset
    val_pred = classifier.predict(validationSet)

    # Insert predictions into validation dataframe
    validation_y.insert(1, "Pred", val_pred, True)
    
    # Round continuous predictions into true or false (0 or 1)
    validation_y['Pred'] = validation_y['Pred'].apply(lambda x: round(x))
    
    y_true = validation_y['human']
    y_pred = validation_y['Pred']
    
    getMetrics(y_true, y_pred)
    
    return classifier

In [9]:
# get a prediction from the ensamble model which creates a composite pred as a result of the output of the 
# textModel and standard model
def ensambleModel(textModel, model, predictionData):
    preppedTextData = prepDataForTextModel(predictionData)
    textPrediction = textModel.predict(preppedTextData)
    preppedModelData = prepDataForModel(predictionData)
    modelPrediction = model.predict(preppedModelData)
    
    compositePred = []
    
    for index in range(len(textPrediction)):
        compositePred.append(abs((textPrediction[index] + modelPrediction[index])/2))
        
    return compositePred
    

In [10]:
trainingSet = munge()

In [11]:
textModel = trainTextModel(trainingSet)

Index(['id', 'rawText', 'cleanText', 'emojiCount', 'followers', 'following',
       'hashtagCount', 'urlCount', 'tweetCount', 'verified', 'human',
       'embededText'],
      dtype='object')


100%|██████████████████████████████████████| 2146/2146 [00:05<00:00, 426.53it/s]
/Applications/anaconda3/envs/PoliticalBotAnalysis/lib/python3.9/site-packages/lightgbm/callback.py:183: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.83519
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.863944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 0.887039
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.894053
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 0.905295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's auc: 0.90458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's auc: 0.91149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	valid_0's auc: 0.916314
[LightGBM] [Wa

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[106]	valid_0's auc: 0.924028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[107]	valid_0's auc: 0.923849
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[108]	valid_0's auc: 0.924474
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[109]	valid_0's auc: 0.924355
[110]	valid_0's auc: 0.924504
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[111]	valid_0's auc: 0.924564
[112]	valid_0's auc: 0.925636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[113]	valid_0's auc: 0.924117
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[114]	valid_0's auc: 0.924057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[115]	valid_0's auc: 0.924534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[116]	valid_0's auc: 0.92

[205]	valid_0's auc: 0.924713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[206]	valid_0's auc: 0.923789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[207]	valid_0's auc: 0.923819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[208]	valid_0's auc: 0.924028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[209]	valid_0's auc: 0.924266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[210]	valid_0's auc: 0.924117
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	valid_0's auc: 0.924266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[212]	valid_0's auc: 0.924028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	valid_0's auc: 0.923819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[214]	valid_0's auc: 0.923343
[LightGBM] [Warning] No f

...

[662]	valid_0's auc: 0.931652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[663]	valid_0's auc: 0.931741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[664]	valid_0's auc: 0.931681
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[665]	valid_0's auc: 0.931741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[666]	valid_0's auc: 0.931652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[667]	valid_0's auc: 0.931681
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[668]	valid_0's auc: 0.931681
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[669]	valid_0's auc: 0.931473
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[670]	valid_0's auc: 0.931503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[671]	valid_0's auc: 0.931473
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[764]	valid_0's auc: 0.935702
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[765]	valid_0's auc: 0.935702
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[766]	valid_0's auc: 0.935762
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[767]	valid_0's auc: 0.935762
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[768]	valid_0's auc: 0.935702
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[769]	valid_0's auc: 0.935583
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[770]	valid_0's auc: 0.935583
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[771]	valid_0's auc: 0.935523
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[772]	valid_0's auc: 0.935553
[LightGBM] [Warning] No further splits with positive ga

[864]	valid_0's auc: 0.936506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[865]	valid_0's auc: 0.936506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[866]	valid_0's auc: 0.936953
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[867]	valid_0's auc: 0.937042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[868]	valid_0's auc: 0.937072
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[869]	valid_0's auc: 0.936804
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[870]	valid_0's auc: 0.936834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[871]	valid_0's auc: 0.936863
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[872]	valid_0's auc: 0.936953
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[873]	valid_0's auc: 0.937072
[LightGBM] [Warning] No f

[963]	valid_0's auc: 0.937876
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[964]	valid_0's auc: 0.937787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[965]	valid_0's auc: 0.937846
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[966]	valid_0's auc: 0.937906
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[967]	valid_0's auc: 0.937906
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[968]	valid_0's auc: 0.937936
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[969]	valid_0's auc: 0.937965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[970]	valid_0's auc: 0.937995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[971]	valid_0's auc: 0.938025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[972]	valid_0's auc: 0.938084
[LightGBM] [Warning] No f

/var/folders/k4/dhwf5dxn5x96j8q55gsl15g80000gn/T/ipykernel_44291/803779095.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_y['Pred'] = validation_y['Pred'].apply(lambda x: round(x))


In [12]:
model = trainModel(trainingSet)

/Applications/anaconda3/envs/PoliticalBotAnalysis/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Applications/anaconda3/envs/PoliticalBotAnalysis/lib/python3.9/site-packages/lightgbm/callback.py:183: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


Index(['emojiCount', 'followers', 'following', 'hashtagCount', 'urlCount',
       'tweetCount', 'verified'],
      dtype='object')
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001283 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.940225
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.982251
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 0.977149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.974554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 0.985791
[6]	valid_0's auc: 0.988807
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's auc: 0.987997
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[113]	valid_0's auc: 0.991898
[114]	valid_0's auc: 0.992198
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[115]	valid_0's auc: 0.992138
[116]	valid_0's auc: 0.991838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[117]	valid_0's auc: 0.991748
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[118]	valid_0's auc: 0.991658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[119]	valid_0's auc: 0.991688
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[120]	valid_0's auc: 0.991718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[121]	valid_0's auc: 0.991718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[122]	valid_0's auc: 0.991718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[123]	valid_0's auc: 0.99

[204]	valid_0's auc: 0.991118
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[205]	valid_0's auc: 0.991088
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[206]	valid_0's auc: 0.991118
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[207]	valid_0's auc: 0.991118
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[208]	valid_0's auc: 0.991148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[209]	valid_0's auc: 0.991148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[210]	valid_0's auc: 0.991148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	valid_0's auc: 0.991178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[212]	valid_0's auc: 0.991028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	valid_0's auc: 0.990968
[LightGBM] [Warning] No f

[344]	valid_0's auc: 0.989317
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[345]	valid_0's auc: 0.989437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[346]	valid_0's auc: 0.989317
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[347]	valid_0's auc: 0.989347
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[348]	valid_0's auc: 0.989587
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[349]	valid_0's auc: 0.989497
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[350]	valid_0's auc: 0.989557
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[351]	valid_0's auc: 0.989617
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[352]	valid_0's auc: 0.989497
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[353]	valid_0's auc: 0.989797
[LightGBM] [Warning] No f

...

[650]	valid_0's auc: 0.990608
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[651]	valid_0's auc: 0.990788
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[652]	valid_0's auc: 0.990758
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[653]	valid_0's auc: 0.990788
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[654]	valid_0's auc: 0.990698
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[655]	valid_0's auc: 0.990608
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[656]	valid_0's auc: 0.990698
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[657]	valid_0's auc: 0.990698
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[658]	valid_0's auc: 0.990548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[659]	valid_0's auc: 0.990578
[LightGBM] [Warning] No f

[736]	valid_0's auc: 0.990758
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[737]	valid_0's auc: 0.990788
[738]	valid_0's auc: 0.990818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[739]	valid_0's auc: 0.990728
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[740]	valid_0's auc: 0.990788
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[741]	valid_0's auc: 0.990788
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[742]	valid_0's auc: 0.990638
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[743]	valid_0's auc: 0.990608
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[744]	valid_0's auc: 0.990488
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[745]	valid_0's auc: 0.990488
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[746]	valid_0's auc: 0.99

[856]	valid_0's auc: 0.990038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[857]	valid_0's auc: 0.990038
[858]	valid_0's auc: 0.990038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[859]	valid_0's auc: 0.990038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[860]	valid_0's auc: 0.990068
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[861]	valid_0's auc: 0.990068
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[862]	valid_0's auc: 0.990218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[863]	valid_0's auc: 0.990218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[864]	valid_0's auc: 0.990248
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[865]	valid_0's auc: 0.990158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[866]	valid_0's auc: 0.99

[960]	valid_0's auc: 0.990218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[961]	valid_0's auc: 0.990188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[962]	valid_0's auc: 0.990278
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[963]	valid_0's auc: 0.990188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[964]	valid_0's auc: 0.990218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[965]	valid_0's auc: 0.990188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[966]	valid_0's auc: 0.990158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[967]	valid_0's auc: 0.990188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[968]	valid_0's auc: 0.990158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[969]	valid_0's auc: 0.990158
[LightGBM] [Warning] No f

/var/folders/k4/dhwf5dxn5x96j8q55gsl15g80000gn/T/ipykernel_44291/2036535791.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_y['Pred'] = validation_y['Pred'].apply(lambda x: round(x))


In [13]:
# validation set from training to demonstrate how ensamble model works
predictionSet = trainingSet.tail(300)

In [14]:

returnValues = ensambleModel(textModel, model, predictionSet)

100%|████████████████████████████████████████| 300/300 [00:00<00:00, 417.83it/s]


In [35]:
# round predictions into 1 and 0s and get evaluation metrics
roundedReturnValues = [round(x) for x in returnValues]
getMetrics(predictionSet.human.values, roundedReturnValues)
tn, fp, fn, tp = confusion_matrix(predictionSet.human.values, roundedReturnValues).ravel()
print("True Negative: %s \n False Negative: %s \n True Positive: %s \n False Positive: %s" % (tn, fp, fn, tp))

[[187   6]
 [ 12  95]]
Accuracy: 0.94
Precision: 0.9405940594059405
Recall: 0.8878504672897196
True Negative: 187 
 False Negative: 6 
 True Positive: 12 
 False Positive: 95


In [16]:
# save the text model to disk
filename = 'text_model.sav'
pickle.dump(textModel, open(filename, 'wb'))
print("Text Model Pickled")

Text Model Pickled


In [17]:
# save the model to disk
filename = 'standard_model.sav'
pickle.dump(model, open(filename, 'wb'))
print("Standard Model Pickled")

Standard Model Pickled
